# TP 1. Digital House Grupo N7

Desafío 1. Análisis exploratorio de un dataset de
precios de propiedades

**Objetivos:**

*   Efectuar una limpieza del dataset provisto. Particularmente, deberá diseñar estrategias para lidiar con los datos perdidos en ciertas variables.
*   Realizar un análisis descriptivo de las principales variables.
* Crear nuevas columnas a partir de las características dadas que puedan tener valor
predictivo.


# Setup Inicial

In [ ]:
#descargo archivos
! gdown 1Ui7dofcyVVJlqPSqnJjmufPKnZxw5ZAj #properatti.csv
! gdown 1I2V15Sf2mUqXQUQvExfvzipDxnWMqvXM #provincia.shx
! gdown 1Im6hNqTDb1zaCX4Lo9czGaMmqgzvDj9S #ar.csv
! gdown 1RCIHoEfk07OiSZw5PtOxVdj4aiHxQJpC #provincia.shp
! gdown 1EcVqccbr0qdJtuyzOHBRtSkkJZlxf6_Z #provincia.prj
! gdown 1r3ElDulmkTvdGwL6zuIWLZ08C8bM4kqw #provincia.dbf
! gdown 11VXNu7X_BIfB-2ZtStlxQmHoL_OBdafk #provincia.cpg

In [ ]:
#instalo dependencias
!pip install --upgrade geopandas pyshp shapely descartes
!pip install --upgrade unidecode
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
from unidecode import unidecode
import seaborn as sns
from importlib.metadata import version
version('unidecode')

In [ ]:
#Carga del dataframe
properatti_df_original=pd.read_csv('/content/properatti.csv', sep=",")
properatti_df=properatti_df_original.copy()

# Análisis Inicial del dataset

In [ ]:
#Configuración para ver todas las columnas del dataframe sin el "..."
pd.options.display.max_columns = None
#Se muestra el head del df
properatti_df.head()

<h3>Se exploraron las distintas columnas del Dataframe con el atributo  Dataframe.dtypes, el tamaño del dataset(shape), la cantidad de valores por cada columna(count) y </h3>

> * Aqui se puede observar que las columnas tienen el tipo de dato esperado. Float64 para valores numericos y objects para columnas de tipo string.
> * Hay 121220 fila pero no todas las columnas tienen la misma cantidad de valores.
> * No existen filas duplicadas.




In [ ]:
properatti_df.dtypes

In [ ]:
properatti_df.shape

In [ ]:
properatti_df.count()

In [ ]:
properatti_df[properatti_df.duplicated].count()

<h2>Evaluación de los valores nulos en el dataset</h2>

In [ ]:
porcentaje_de_nulos = properatti_df.isna().sum()/(properatti_df.shape[0])*100
porcentaje_de_nulos.round(2)

Representación gráfica de los valores nulos

In [ ]:
lista=properatti_df.columns.tolist()
valores=porcentaje_de_nulos.round(2).tolist()

In [ ]:
fig, ax = plt.subplots()
ax.bar(lista, valores, tick_label= lista)

ax.set(xlabel = "Columna", ylabel = "Porcentaje de nulos y", title = "Porcentaje de nulos en cada columna")
plt.xticks(rotation=45, ha="right")
plt.gcf().set_size_inches(20, 6)
plt.show()

Evaluar algunos datos no nulos o con bajo porcentaje de nulos

In [ ]:
properatti_df['operation'].value_counts()

In [ ]:
properatti_df['properati_url'].value_counts()

In [ ]:
properatti_df['image_thumbnail'].value_counts()

In [ ]:
properatti_df['property_type'].value_counts()


<h2> Definición de Estrategias en la Limpieza de Datos</h2>
<h4>Se puede observar que las columnas operation, property_type, place_with_parent_names, country_name, state_name, properati_url y title tienen valores no nulos</h4>

<h3>Eliminación de datos </h3>

><h4><u>image_thumbnail</u></h4>
><h5>quitaremos, no es util </h5>

><h4><u>properati_url</u></h4>
><h5>quitaremos, no es util </h5>

><h4><u>Operation</u></h4>
><h5>El unico tipo de operacion es 'sell' por lo tanto se eliminara del dataset</h5>

<h3>Datos de soporte para otros cálculos</h3>

><h4><u>property_type</u></h4>
><h5>Veremos los distintos tipos de propiedades, normalizaremos de ser necesario. Esta columna es importante ya que luego para analisis de precio seguramente sea necesario discriminar</h5>

><h4><u>description</u></h4>
><h5>utilizaremos para buscar info de otras columnas</h5>

><h4><u>title</u></h4>
><h5>utilizaremos para buscar info de otras columnas</h5>


<h3>Completar Datos faltantes </h3>

><h4><u>geonames_id, lat-lon, lat y lon</u></h4>
><h5>Limpiaremos y utilizaremos esta informacion para visualizar la ubicacion de las propiedades. La informacion de la geografia se complementa con informacion proveniente de un dataframe extra (DATA_GEONAMES) y dos mapas de Argentina, uno con division politica y otro continental.</h5>

><h4><u>place_with_parent_names</u></h4>
><h5>Normalizaremos esta columna, para obtener informacion acerca de la ubicacion en terminos de provincia, ciudad y barrio. Dicha informacion se guardara en columnas nuevas.</h5>

><h4><u>price, currency, price_aprox_local_currency,price_aprox_usd</u></h4>

><h5>Normalizaremos en una nueva columna con precio aproximado en usd, utilizando el valor de la columna price_aprox_usd, intentando extraer via regex de la columna descripcion, o convirtiendo los valores en caso de que esten en otra moneda</h5>

><h4><u>surface_total_in_m2</u></h4>
<h5>Crearemos una nueva columna con los valores de la columna original y extrayendo los faltantes que se puedan encontrar, mediante una regex, en las columnas descripción, title y url.</h5>

><h4><u>surface_covered_in_m2</u></h4>
><h5>Crearemos una nueva columna con los valores de la columna original y extrayendo los faltantes que se puedan encontrar, mediante una regex, en las columnas descripción, title y url.</h5>

><h4><u>price_usd_per_m2</u></h4>
<h5>Representa sup_total / precio. calcularemos bonde haga falta con los valores de las columnas correspondientes</h5>

><h4><u>price_per_m2</u></h4>
><h5>Utilizaremos la column price_usd_per_m2, pero utilizaremos esta si podemos para llenar donde falte price_usd_per_m2</h5>

><h4><u>Floor</u></h4>
<h5>Veremos si es relevante, trataremos completar con regex </h5>

><h4><u>Rooms</u></h4>
><h5>Buscaremos con regex, sino veremos correlacion con surface_covered_in_m2 para copmletar</h5>

><h4><u>Expenses</u></h4>
><h5>Veremos si es relevante, trataremos completar con regex </h5>








# Eliminación de Datos

Eliminar datos no útiles

In [ ]:
#Se eliminan las columnas mencionadas en la sección anterior.

properatti_df.drop('operation', axis=1, inplace=True)
properatti_df.drop('image_thumbnail', axis=1, inplace=True)
properatti_df.drop('properati_url', axis=1, inplace=True)

Eliminar datos erróneos
>Eliminar pubicaciones de pozos

In [ ]:
#Normalizacion de los datos
properatti_df['description'] = properatti_df['description'].str.lower()
properatti_df['title'] = properatti_df['title'].str.lower()

In [ ]:
# Se buscan publicaciones que tengan palabras que sugieran publicaciones de tipo lote de multiples propiedades, las cual no nos interesa ya que contaminan el dataset
search_strings = ["unidades", "departamentos", " uds", "deptos"]

# Creo mascara para detectar si la columna title tiene alguna de mis palabras a buscar
mask_search_string = properatti_df['title'].str.contains('|'.join(search_strings))

# Filtrar el data frame
multiple_filter_df = properatti_df[mask_search_string]

# Se imprimen las primeras 100 descripciones de entries que cumplan mi mascara para analizarlas
row_iterator=0
for index, row in multiple_filter_df.iterrows():
  if row_iterator>100:
    break
  print(f"Index: {index}\nTitle: {row['title']}\nDescription: {row['description']}\nRooms: {row['rooms']}\nFloor: {row['floor']}\nPrice: {row['price_aprox_usd']}\n\n")
  row_iterator+=1

In [ ]:
multiple_filter_df.shape[0]

><h4>Vemos que la mayoria de estas entries son efectivamente publicaciones de pozo, o de multiples departamentos a la vez, y al ver que son 1313, decidimos retirarlas del Dataframe sobre el cual trabajaremos</h4>

In [ ]:
properatti_df = properatti_df[~mask_search_string]
print("entries restantes:", properatti_df.shape[0])

>Evaluar valores de surface_total_in_m2,surface_covered_in_m2,price_aprox_usd y price erróneos que tienen valores menores a un threshold lógico.

In [ ]:
print(properatti_df[properatti_df['surface_total_in_m2']<5].shape[0])
print(properatti_df[properatti_df['surface_covered_in_m2']<2].shape[0])
print(properatti_df[properatti_df['price_aprox_usd']<100].shape[0])
print(properatti_df[properatti_df['price']<100].shape[0])

Eliminar los valores que esten en dentro del threshold definido

In [ ]:
#convierto con una mascara y una funcion a nan
def replace_value_with_nan(x,threshold):
    return np.nan if x < threshold else x

properatti_df['surface_total_in_m2'] = properatti_df['surface_total_in_m2'].apply(replace_value_with_nan, args=(5,))
properatti_df['surface_covered_in_m2'] = properatti_df['surface_covered_in_m2'].apply(replace_value_with_nan, args=(5,))
properatti_df['price_aprox_usd'] = properatti_df['price_aprox_usd'].apply(replace_value_with_nan, args=(100,))
properatti_df['price'] = properatti_df['price'].apply(replace_value_with_nan, args=(100,))

# Completar datos faltantes

In [ ]:
def calculate_correlation(dataframe, column1, column2, lower_bound1=None, lower_bound2=None, upper_bound1=None, upper_bound2=None):
    #Si no paso los limites, los calculo de acuerdo a la formula tradicional
    if lower_bound1 is None or upper_bound1 is None:
        Q1 = dataframe[column1].quantile(0.25)
        Q3 = dataframe[column1].quantile(0.75)
        IQR = Q3 - Q1
        if lower_bound1 is None:
            lower_bound1 = Q1 - 1.5 * IQR
        if upper_bound1 is None:
            upper_bound1 = Q3 + 1.5 * IQR

    if lower_bound2 is None or upper_bound2 is None:
        Q1 = dataframe[column2].quantile(0.25)
        Q3 = dataframe[column2].quantile(0.75)
        IQR = Q3 - Q1
        if lower_bound2 is None:
            lower_bound2 = Q1 - 1.5 * IQR
        if upper_bound2 is None:
            upper_bound2 = Q3 + 1.5 * IQR

    #Filtro outliers
    dataframe = dataframe[(dataframe[column1] >= lower_bound1) & (dataframe[column1] <= upper_bound1)]
    dataframe = dataframe[(dataframe[column2] >= lower_bound2) & (dataframe[column2] <= upper_bound2)]

    #Calculo la correlacion
    correlation = dataframe[[column1, column2]].corr().iloc[0,1]

    return correlation


#correlacion en grupos
def calculate_correlation_grouped(df, column1, column2, groupby_column,
                           lower_bound1=None, lower_bound2=None,
                           upper_bound1=None, upper_bound2=None):

    # Agrupo segun la columna pasada como groupby_column
    grouped = df.groupby(groupby_column)

    correlation_list = []
    for name, group in grouped:

        #Si no paso los limites, los calculo de acuerdo a la formula tradicional
        if lower_bound1 is None or upper_bound1 is None:
            Q1 = group[column1].quantile(0.25)
            Q3 = group[column1].quantile(0.75)
            IQR = Q3 - Q1
            if lower_bound1 is None:
                lower_bound1 = Q1 - 1.5 * IQR
            if upper_bound1 is None:
                upper_bound1 = Q3 + 1.5 * IQR

        if lower_bound2 is None or upper_bound2 is None:
            Q1 = group[column2].quantile(0.25)
            Q3 = group[column2].quantile(0.75)
            IQR = Q3 - Q1
            if lower_bound2 is None:
                lower_bound2 = Q1 - 1.5 * IQR
            if upper_bound2 is None:
                upper_bound2 = Q3 + 1.5 * IQR

        # Filtro outliers
        group = group[(group[column1] > lower_bound1) & (group[column1] < upper_bound1) &
                      (group[column2] > lower_bound2) & (group[column2] < upper_bound2)]

        # Chequeo si el grupo tiene informacion suficiente para calular la correlacion
        if group.shape[0] > 1 and group[column1].nunique() > 1 and group[column2].nunique() > 1:
            correlation = group[[column1, column2]].corr().iloc[0,1]
            correlation_list.append(correlation)

    # chequeo si tengo correlaciones en mi lista, luego calculo la media
    if len(correlation_list) > 0:
        return np.mean(correlation_list)
    else:
        return None

In [ ]:
#funcion para imputar dato en columna column2 en base a la media para segun column2 cuando existe
def get_column2_with_column1(df,column1,column2):

    #filtro las entries con column1 valida
    valid_rows = df[df[column1].notnull()]

    #agrupo por column1 y obtengo la media de column2
    mean_column2_by_column1 = valid_rows.groupby(column1)[column2].mean()

    #recorro mi df y asigno valor a column2 cuando no tiene valor
    for index, row in df.iterrows():
        if pd.isna(row[column2]):
            column1_value = row[column1]
            if column1_value in mean_column2_by_column1.index:
                mean_column2 = mean_column2_by_column1[column1_value]
                # Chequeo si la media != NaN, y asigno valor teniendo en cuanta que la media puede ser un float, y dando mas probabilidad al entero mas cercano, y menor al mas lejano, si mean = 3.1, tendra mas prob
                # de imputar 3, que de imputar 4
                if not np.isnan(mean_column2):
                    probability = mean_column2 - int(mean_column2)
                    rounded_column2 = np.random.choice([np.floor(mean_column2), np.ceil(mean_column2)], p=[1 - probability, probability])
                    df.at[index, column2] = rounded_column2
                else:
                    #si la media es NaN, se imputa NaN
                    df.at[index, column2] = mean_column2
    return df

# Ubicaciones (Juan)

In [ ]:
# En el dataframe properatti_df, la informacion relevante para limpiar relacionada a la geografia esta distribuida en 4 columnas:
  # geonames_id, lat,	lon, y	lat-lon

# Se limpiarán los datos relacionados a la geografia, con el objetivo de:
  # poder graficar cada propiedad como un punto en un mapa. El mapa puede ser de todo el pais, por provincia o por region.
  # poder utilizar datos geograficos para realizar analisis exploratorios (ej., histograma de precios por ciudad, provincia, etc.)

# la estrategia para limpiar la geografia va a ser:
  # Imputar en la medida de lo posible la informacion de ubicación geografica (latitud y longitud) sobre las columnas 'lat' y 'lon' para todas las propiedades.
  # cuando no haya informacion en estos campos, tratar de recolectarla del geodataframe 'DATA_GEONAMES' (https://www.geonames.org/countries/AR/argentina.html)
    # Esto se hara de dos maneras:
      # la primera, trayendo la informacion de latitud y longitud de DATA_GEONAMES que coincida con el identificador de geonames de la propiedad.
      # Cuando esto no sea posible, se entrecruzará la informacion de ciudad o provincia de cada propiedad con la del dataframe DATA_GEONAMES, para traer la informacion de lat y lon correspondiente.
      # (esto es una aproximacion, ya que para cada propiedad, los valores de latitud y longitud seran los de la ciudad o provincia correspondiente)
    # Para esto, se limpiará la columna 'place_with_parent_names', y la información se distribuirá en cuatro columnas nuevas:
      # provincia, partido, ciudad y barrio.

properatti_df[['title','geonames_id','lat','lon', 'lat-lon']]

In [ ]:
# importo el Mapa de Argentina CON info por provincias

data_location = "/content/provincia.shx"
mapa_argentina = gpd.read_file(data_location)

# importo el Mapa (2) de Argentina SIN info por provincias

mundo = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
mapa_argentina_2 = mundo[mundo.name == "Argentina"]

mapa_argentina.head(24)

In [ ]:
# este mapa trae los poligonos de cada provincia.

mapa_argentina.plot()

In [ ]:
# este mapa no trae info de provincias (mas practico para ploteo general de todas las propiedades)

mapa_argentina_2.plot()

In [ ]:
# Traemos el geodataframe 'data_geonames'

location_geonames = '/content/AR.csv'
data_geonames = gpd.read_file(location_geonames, sep='\t')

print(data_geonames.shape)
data_geonames.head(5)

In [ ]:
# Se hacen algunos ajustes al dataframe DATA_GEONAMES:
# Primero, convierto a float64 la columna 'GEONAMEID' que es un string. Dicha columna se usara para matchear con 'geonames_id' en properatti_df (float64).

data_geonames['GEONAMEID'] = pd.to_numeric(data_geonames['GEONAMEID'], errors='coerce').astype(float)

In [ ]:
# Segundo, utilizo la informacion de 'TIMEZONE' para crear una columna llamada 'provincia'. Dicha columna se usara para matchear con 'provincia' en properatti_df (object)).

pattern = re.compile(r'/([^/]+)$')

def cambiar_nombre(timezone):
    match = re.search(pattern, timezone)
    return match.group(1).replace('_', ' ') if match else None

data_geonames['provincia'] = data_geonames['TIMEZONE'].apply(lambda x: cambiar_nombre(x))
data_geonames.head(5)

In [ ]:
# chequeo de valores nulos en las columnas de la geografia:

mask_lat_lon = np.logical_and(properatti_df['lat'].isnull(), properatti_df['lon'].isnull())
mask_latlon = properatti_df['lat-lon'].isnull()
mask_geonames = properatti_df['geonames_id'].isnull()
combined_mask = mask_lat_lon & mask_geonames


geografia_filtered_df = properatti_df.loc[mask_lat_lon]
print("nulls en 'lat' y 'lon': " f"{geografia_filtered_df.shape}")
geografia_filtered_df = properatti_df.loc[mask_latlon]
print("nulls en 'lat-lon': " f"{geografia_filtered_df.shape}")
# 'lat', 'lon' y 'lat-lon' tienen la misma info

geografia_filtered_df = properatti_df.loc[mask_geonames]
print("nulls en 'geonames_id': " f"{geografia_filtered_df.shape}")
geografia_filtered_df = properatti_df.loc[combined_mask]
print("ninguna informacion: " f"{geografia_filtered_df.shape}")




In [ ]:
# Estado de los valores nulos al comienzo del proceso.

nulls_df = properatti_df[['lat', 'lon', 'lat-lon', 'geonames_id']].isnull()

# Plotear los valores nulos en un heatmap abarcando todo el dataframe.

plt.figure(figsize=(10, 6))
sns.heatmap(nulls_df, cmap='viridis', cbar=False, yticklabels=False)
plt.title('Valores Nulos en Columnas de Geografía')
plt.show()

# Se ve claramente que 'lat', 'lon' y 'lat-lon' son redundantes, es decir tienen la misma informacion. La diferencia es que 'lat-lon' tiene los valores 'lat' y 'lon' en forma de lista.

In [ ]:
# Un pantallazo a la columna 'place_with_parent_names' muestra que es en principio la informacion sobre la provincia (cuando se trata de Buenos Aires, tambien hay informacion sobre el partido), ciudad, y en algunos casos barrio.
# Esta informacion me va a servir para cruzar datos con DATA GEONAMES, cuando no haya informacion de 'lat' y 'lon'.

properatti_df['place_with_parent_names'].value_counts()

In [ ]:
# Chequeo si falta algun dato en 'place_with_parent_names'

properatti_df.place_with_parent_names.isnull().sum()

# Todos tienen algun tipo de informacion de lugar(pais provincia ciudad)

In [ ]:
# Filtro los acentos en DATA_GEONAMES y en properatti_df, para que si hay que matchear, no haya diferencias por acentos.

columns_2 = ['place_with_parent_names']
columns_1 = ['NAME', 'ASCIINAME', 'provincia']

def remove_accents(text):
    return unidecode(text) if pd.notna(text) else text

properatti_df[columns_2] = properatti_df[columns_2].applymap(remove_accents)
data_geonames[columns_1] = data_geonames[columns_1].applymap(remove_accents)

In [ ]:
# Asigno la informacion de 'place_with_parent_name' a tres nuevas columnas: 'provincia', 'ciudad' y 'barrio'

pattern = re.compile(r'\|(?P<pais>[^|]+)\|(?P<provincia>[^|]+)(?:\|(?P<ciudad>[^|]+))?(?:\|(?P<barrio>[^|]+))?')

def extract_parts(location_info):
    match = pattern.match(location_info)
    return {'provincia': match.group('provincia') if match else None, 'ciudad': match.group('ciudad') if match else None, 'barrio': match.group('barrio') if match else None}

properatti_df[['provincia', 'ciudad', 'barrio']] = properatti_df['place_with_parent_names'].apply(
    lambda x: pd.Series(extract_parts(x))
)




In [ ]:
# Mando a Capital Federal adentro de la Provincia de Buenos Aires (por razones geograficas -no administrativas-)

def clean_capital(row):
    if row['provincia'] == 'Capital Federal':
        row['provincia'] = 'Buenos Aires'
        row['barrio'] = row['ciudad']
        row['ciudad'] = 'Ciudad Autonoma de Buenos Aires'
    return row

properatti_df = properatti_df.apply(clean_capital, axis=1)

In [ ]:
# resultado de limpiar 'place_with_parent_names'

properatti_df[['title','provincia','ciudad','barrio']].head(10)

In [ ]:
# Vemos que cuando la provincia es Buenos Aires, el dato nos queda sucio, con informacion sobre el partido.
properatti_df['provincia'].value_counts()

In [ ]:
# Uso regex para limpiar Buenos Aires, enviando la informacion subsiguiente a una nueva columna 'partido'
pattern = re.compile(r'(?P<provincia>Bs\.As\.|Buenos Aires)\s(?P<partido>.+)')

def match_location(location_info):
    if pd.isna(location_info):  # Chequea valores faltantes
        return pd.Series({'provincia': None, 'partido': None})

    match = pattern.match(location_info)
    if match:
        provincia = 'Buenos Aires'
        partido = match.group('partido')
        return pd.Series({'provincia': provincia, 'partido': partido})
    else:
        return pd.Series({'provincia': location_info, 'partido': None})


properatti_df[['provincia', 'partido']] = properatti_df['provincia'].apply(match_location)

In [ ]:
# Vemos que ahora, el dato Buenos Aires aparece unificado, y el resto de la informacion aparece en 'partido'
properatti_df[['provincia','partido']].value_counts()

In [ ]:
# Pantallazo al campo 'ciudad' correspondiente a 'provincia' == Buenos Aires
properatti_df[properatti_df['provincia'] == 'Buenos Aires']['ciudad'].value_counts()

In [ ]:
# Primer paso para imputar valores nulos en 'lat' y 'lon':

# Matchea los valores de 'geonames_id' en el dataframe 'data_geonames'
# cuando los encuentra, se trae el valor de las columnas 'LATITUDE' y 'LONGITUDE'

def normalize_geonameid(row):
    if pd.isnull(row['lat']) and pd.isnull(row['lon']):
        geoname_id = row['geonames_id']
        if geoname_id in data_geonames['GEONAMEID'].values:
            matching_row = data_geonames[data_geonames['GEONAMEID'] == geoname_id]
            if not matching_row.empty:
                row.loc['lat'] = matching_row['LATITUDE'].iloc[0]
                row.loc['lon'] = matching_row['LONGITUDE'].iloc[0]

    return row

properatti_df = properatti_df.apply(normalize_geonameid, axis=1)

In [ ]:
properatti_df[['lat', 'lon']].isnull().sum()

In [ ]:
# Estado de los valores nulos al final del primer paso

nulls_df = properatti_df[['lat', 'lon', 'geonames_id']].isnull()
plt.figure(figsize=(10, 6))
sns.heatmap(nulls_df, cmap='viridis', cbar=False, yticklabels=False)
plt.title('Valores Nulos en Columnas de Geografía')
plt.show()

# La reduccion de nulos es considerable.

In [ ]:
# Segundo paso:
# Entrecruzo las columnas 'ciudad' y 'NAME' en DATA_GEONAMES (tiene informacion de ciudad).
# Si hay matches, me traigo los valores 'LATITUDE' y 'LONGITUDE' de data_geonames.

def find_ciudad_province_in_data_geonames(row):
    if pd.isnull(row['lat']) and pd.isnull(row['lon']):
        ciudad = row['ciudad']
        matching_rows = data_geonames[data_geonames['NAME'].isin([ciudad])]

        if not matching_rows.empty:
            matching_row = matching_rows.iloc[0]
            row['lat'] = pd.to_numeric(matching_row['LATITUDE']).astype(float)
            row['lon'] = pd.to_numeric(matching_row['LONGITUDE']).astype(float)

    return row

properatti_df = properatti_df.apply(find_ciudad_province_in_data_geonames, axis=1)

In [ ]:
# La reducción de nulos es significativa.

properatti_df[['lat', 'lon']].isnull().sum()

In [ ]:
# Tercer paso:
# Entrecruzo las columnas 'provincia' (properatti_df) y 'provincia' en DATA_GEONAMES (campo creado a partir de 'TIMEZONE' que tiene info de provincias).
# Si hay matches, me traigo los valores 'LATITUDE' y 'LONGITUDE' de data_geonames.

def find_ciudad_province_in_data_geonames(row):
    if pd.isnull(row['lat']) and pd.isnull(row['lon']):
        provincia = row['provincia']
        matching_rows = data_geonames[data_geonames['provincia'].isin([provincia])]

        if not matching_rows.empty:
            matching_row = matching_rows.iloc[0]
            row['lat'] = pd.to_numeric(matching_row['LATITUDE']).astype(float)
            row['lon'] = pd.to_numeric(matching_row['LONGITUDE']).astype(float)

    return row

properatti_df = properatti_df.apply(find_ciudad_province_in_data_geonames, axis=1)

In [ ]:
# No hay mas nulos, por lo tanto todos las propiedades han sido imputadas con algun dato relacionado a su ubicación geográfica.

properatti_df[['lat', 'lon']].isnull().sum()

In [ ]:
# creo un geoDataFrame con la info geografica
geometry = gpd.points_from_xy(pd.to_numeric(properatti_df.lon), pd.to_numeric(properatti_df.lat))


properatti_df = gpd.GeoDataFrame(properatti_df, geometry=geometry)

# outlier corregido (aparentemente es un error en el dataframe 'DATA_GEONAMES')

properatti_df.at[9761, 'geometry'] = Point(-64.00000, -32.00000)


In [ ]:
# Objetivo 1: Plot de todas las propiedades:

properatti_df_final_geografia = properatti_df[['title','place_with_parent_names','provincia','ciudad','barrio','partido','geometry']]
print("Shape: " , properatti_df_final_geografia.shape)
print("total nulos: ", properatti_df_final_geografia['geometry'].isnull().sum())

# Mapa de Argentina
ax = mapa_argentina_2.plot(color='white', edgecolor='black')

# Sobre las ciudades superpone el mapa del país
properatti_df.plot(ax=ax, color='red', markersize=0.05)

plt.show()



In [ ]:
# Objetivo 1.1: Plot por provincia. En este caso, (ej.) San Luis.

mapa_san_luis = mapa_argentina[mapa_argentina.fna == "Provincia de San Luis"]

  # Para plotear, filtro los puntos que estan dentro de 'mapa_san_luis'

contained_points = properatti_df[properatti_df.geometry.within(mapa_san_luis.unary_union)]

# plot del mapa_san_luis
ax = mapa_san_luis.plot(figsize=(10, 10), color='lightgray', edgecolor='black')
contained_points.plot(ax=ax, color='red', marker='o', markersize=5, alpha=0.5)
plt.title('Propiedades en San Luis')
plt.show()

In [ ]:
# Objetivo 2: Histograma de precios en tres barrios de Mar del Plata:

# Barrios específicos en Mar del Plata
barrios = ['Centro', 'Plaza Mitre', 'Chauvin']

# Filtra el DataFrame original para incluir solo las propiedades de esos barrios en Mar del Plata
precios_barrios = properatti_df[(properatti_df['ciudad'] == 'Mar del Plata') & (properatti_df['barrio'].isin(barrios))]

plt.figure(figsize=(12, 8))
sns.violinplot(data=precios_barrios, x='barrio', y='price', palette='Set3')

plt.title('Histograma Superpuesto de Precios en Barrios de Mar del Plata')
plt.xlabel('Precio')
plt.ylabel('Frecuencia')
plt.legend(title='Barrio')
plt.show()



#Limpieza de la columna "surface_total_in_m2"

In [ ]:
#cuento las cantidad de valores nulos de la columa superficie total
df_surface_total = properatti_df['surface_total_in_m2']
print('Nulos en Surface Total:', df_surface_total.isnull().sum())

#calculo el porcentaje de nulos en Superficie Total
print('Procentaje de nulos en Surface Total:', (df_surface_total.isnull().sum()/ properatti_df.shape[0]))

In [ ]:
# separlo las columnas en las que se buscará la información faltante
df_description = properatti_df['description']
df_title = properatti_df['title']


In [ ]:
#me fijo cuantos valores nulos hay en las columnas donde se buscará la información
print('Nulos en Description:', df_description.isnull().sum())

print('Nulos en Title:', df_title.isnull().sum())



In [ ]:
#busco una regex para ver en que descripción/title/url hablan de superficie

surface_pattern = r"(\d+(?:[,.]\d{1,2})?)\s*(m²|metros|m2|mts2|mt2|metros cuadrados|mtrs)"

regex = re.compile(surface_pattern, re.IGNORECASE)
# pruebo si la regex funciona en una descrpción en la que se que hablan de m2

print(regex.search(df_description.iloc[15]))

In [ ]:
#busco una descripción que hable de metros cuadrados para probar la regex

print(df_description.iloc[15])

In [ ]:
# pruebo si la regex funciona en una descrpción en la que se que hablan de m2

print(regex.search(df_description.iloc[15]))

In [ ]:
#pruebo la regex en un texto generado por mi
texto = 'dakfj jdhf urhfsdfh jhdfa fp 245 metros x 80 metros cuadrados djdfhsjkdhf'

prueba = re.findall(surface_pattern, texto)

print(prueba)

In [ ]:
#filtro el data frame
df_filtrado = properatti_df[(properatti_df['surface_total_in_m2'].isnull())]

#veo el tamaño del dataframe filtrado
print(df_filtrado.shape)

In [ ]:
#filtro el dataframe donde la regex encuantra valores y el cuantas encontró
df_filtrado_2 = df_filtrado[
    df_filtrado["description"].str.contains(surface_pattern) |
    df_filtrado["title"].str.contains(surface_pattern)
]

print(df_filtrado_2.shape)

In [ ]:
#defino una función para aplicar la regex
def extract_largest_value(text, regex_pattern):
    matches = re.findall(regex_pattern, text)

    if matches:
        valores_numericos = [float(match[0].replace(',', '.')) for match in matches]
        max_valor = max(valores_numericos)
        return str(max_valor)
    else:
        return ""

In [ ]:
#agrego 3 columnas con los valores encontrados por la regex
print(df_filtrado_2.shape)
df_filtrado_2.loc[:, 'description_short'] = df_filtrado_2['description'].apply(lambda x: extract_largest_value(x, surface_pattern))
df_filtrado_2.loc[:, 'title_short'] = df_filtrado_2['title'].apply(lambda x: extract_largest_value(x, surface_pattern))
print(df_filtrado_2.shape)
print(df_filtrado_2["description_short"])
print(df_filtrado_2["title_short"])

In [ ]:
#defino una función para aplicar otra regex y solo quedamre con los valores numerícos eliminando los m2, metros o metros cuadrados
def extract_number_from_string(string):
    match = re.search(r'(\d+(\.\d+)?)', str(string))
    return float(match.group(1)) if match else pd.NA

#defino una función para juntar los valores obtenidos con los originales del data frame
def fill_null(row):
    if pd.isnull(row['surface_total_in_m2']):
        corresponding_value = df_filtrado_2.loc[df_filtrado_2['index_column'] == row['index_column'], 'description_short'].values
        if len(corresponding_value) > 0:
            return extract_number_from_string(corresponding_value[0])
    return row['surface_total_in_m2']

#Agrego una columna de índice común a ambos DataFrames para machear filas
properatti_df['index_column'] = properatti_df.index
df_filtrado_2['index_column'] = df_filtrado_2.index

#aplico la función y creo la columna 'surface_covered_clean' con los valores
properatti_df['surface_total_clean'] = properatti_df.apply(fill_null, axis=1)

#elimino la columna de índice temporal
properatti_df.drop('index_column', axis=1, inplace=True)
df_filtrado_2.drop('index_column', axis=1, inplace=True)

In [ ]:
#veo cuantos nulos quedaron en la nueva columna y comparo con la orginal
properatti_df['surface_total_clean'] = pd.to_numeric(properatti_df['surface_total_clean'],errors='coerce')
print('Nulos en la Surface_clean:', properatti_df['surface_total_clean'].isnull().sum())
print('Nulos en Surface original', properatti_df['surface_total_in_m2'].isnull().sum())
print('Cantidad de valores agregados:',properatti_df['surface_total_in_m2'].isnull().sum() - properatti_df['surface_total_clean'].isnull().sum())
properatti_df.dtypes

In [ ]:
#redondeo los valores con decimal de "surface_total_in_m2", "surface_clean_in_m2", "surface_covered_in_m2"
properatti_df['surface_total_in_m2'] = properatti_df['surface_total_in_m2'].round()
properatti_df['surface_total_clean'] = properatti_df['surface_total_clean'].round()
properatti_df['surface_covered_in_m2'] = properatti_df['surface_covered_in_m2'].round()

In [ ]:
#calculo la media de las superficies
properatti_df['surface_total_clean'].mean()
print(properatti_df[properatti_df['surface_total_clean']<10]['surface_total_clean'].value_counts())

In [ ]:
#convierto a nan los entries de menos de 10m2
properatti_df['surface_total_clean'] = properatti_df['surface_total_clean'].apply(replace_value_with_nan, args=(5,))

# Rooms

<h1>VEMOS 73830 DATOS NO VALIDOS</h1>

In [ ]:
filtered_df = properatti_df[(properatti_df['rooms'].isnull())]

# Filtramos los datos no validos
print(filtered_df.shape[0])
print(filtered_df['rooms'].isnull().sum())

In [ ]:
pattern_space_between = re.compile(r'(\d+)\s*amb')
pattern_space_after_dot = re.compile(r'\.(?!\s)')

def remove_whitespace_between_number_and_amb(text):
    # defino regex para matchear"<numero> amb" coi nespacio opcional
    result = re.sub(pattern_space_after_dot, '. ', text)
    # Uso re.sub para reemplazar matcheo con  "<numero>amb"
    result = re.sub(pattern_space_between, r'\1amb', result)
    return result

#definimos funcion para reemplazar palabras en determinada columna
def replace_words_in_column(df, column_name, word_replacements):
    for word, replacement in word_replacements:
        df[column_name] = df[column_name].str.replace(rf'{word}', str(replacement))
    return df

# definimos funcion para extraer el mayor numero de mis matches
def extract_biggest_number(text, regex_pattern):
  matches = re.findall(regex_pattern, text)
  if matches:
    numbers = [int(n) for n in matches if int(n)<15]
    if numbers:
      return max(numbers)
    else:
      return ""
  else:
    return ""

In [ ]:
#definimos las palabras a reemplazar
word_replacements = [
    ('un ', 1),
    ('mono', 1),
    ('dos ', 2),
    ('tres ', 3),
    ('cuatro ', 4),
    ('cinco ', 5),
    ('seis ', 6),
    ('siete ', 7),
    ('ocho ', 8),
    ('nueve ', 9)
]
#aplicamos nuestro filtro a title
filtered_df = replace_words_in_column(filtered_df,"title",word_replacements)
#aplicamos la remocion de whitespaces entre numeros y amb
filtered_df['title'] = filtered_df['title'].apply(lambda x: remove_whitespace_between_number_and_amb(x))

#aplicamos nuestro filtro a description
filtered_df = replace_words_in_column(filtered_df,"description",word_replacements)
#aplicamos la remocion de whitespaces entre numeros y amb
filtered_df['description'] = filtered_df['description'].apply(str).apply(lambda x: remove_whitespace_between_number_and_amb(x))


<h1>Normalizamos title para que contenga {numero}amb</h1>

In [ ]:
#pattern para encontrar{numero}amb
pattern = re.compile(r'(\d+)amb')

filtered_df['title_room_number'] = filtered_df['title'].apply(lambda x: extract_biggest_number(x, pattern))
filtered_df['description_room_number'] = filtered_df['description'].apply(lambda x: extract_biggest_number(x, pattern))
print(filtered_df.shape)

In [ ]:
filtered_df['title_room_number'] = pd.to_numeric(filtered_df['title_room_number'], errors='coerce')
filtered_df['description_room_number'] = pd.to_numeric(filtered_df['description_room_number'], errors='coerce')
filtered_df['rooms'] = filtered_df['rooms'].combine_first(filtered_df['title_room_number'])
filtered_df['rooms'] = filtered_df['rooms'].combine_first(filtered_df['description_room_number'])


In [ ]:
value_counts = filtered_df['rooms'].value_counts()
print(value_counts)
print(properatti_df['rooms'].isnull().sum())
properatti_df['rooms_clean'] = properatti_df['rooms'].combine_first(filtered_df['rooms'])
print(properatti_df['rooms_clean'].isnull().sum())
get_column2_with_column1(properatti_df,'surface_covered_in_m2','rooms_clean')
print(properatti_df['rooms_clean'].isnull().sum())
value_counts = properatti_df['rooms_clean'].value_counts()
print(f"Departamentos sin valores en rooms:", len(properatti_df[(properatti_df['rooms_clean'].isnull()) & (properatti_df['property_type'] == 'apartment')]))
#properatti_df[properatti_df['rooms_clean'].isnull()]['property_type'].value_counts()
get_column2_with_column1(properatti_df,'surface_total_clean','rooms_clean')
print(f"Departamentos sin valores en rooms:", len(properatti_df[(properatti_df['rooms_clean'].isnull()) & (properatti_df['property_type'] == 'apartment')]))
properatti_df[properatti_df['rooms_clean'].isnull()]['property_type'].value_counts()
get_column2_with_column1(properatti_df,'rooms_clean','surface_total_clean')
get_column2_with_column1(properatti_df,'rooms_clean','surface_covered_in_m2')
#for index, row in properatti_df[properatti_df['rooms_clean'].isnull()].iterrows():
 # print(f"Index: {index}\nDescription: {row['description']}\nTitle: {row['title']}\nProperty type: {row['property_type']}\nRooms: {row['rooms']}\nClean: {row['surface_total_clean']}\nCovered: {row['surface_covered_in_m2']}\nTotal: {row['surface_total_in_m2']}\n\n")

<h3>Podemos ver que recuperamos aprox 28000 valores de los 73156 que nos faltaban.

# Limpieza de la columna "price"

In [ ]:
#Selecciono las columnas relacionadas con precio como currency, precio, precio aprox usd etc#

precios=properatti_df.loc[:,["price","currency","price_aprox_local_currency","price_aprox_usd","price_usd_per_m2","price_per_m2"]]
precios

In [ ]:
precios['Index'] = range(1, len(precios)+1)
precios["Index"]

In [ ]:
#Calculo cantidad de nulos en precio. siguiente 3 comandos#

nulos=pd.isnull(properatti_df["price"]).sum()
nulos

In [ ]:
porcentaje_nulos=nulos/(properatti_df["price"].shape[0])*100

In [ ]:
porcentaje_nulos

In [ ]:
nulos.dtype

is_notnull_result = precios.notnull()
canti_not_null=is_notnull_result.sum()
print(canti_not_null)


In [ ]:
porcentaje_no_nulos=canti_not_null/(properatti_df.shape[0])*100

In [ ]:
porcentaje_no_nulos

In [ ]:
precios.dtypes

In [ ]:
#corroborar que hay nulos en precios#
precios["price"].head(10)

In [ ]:
#contar cantidad de nulos#
contar_valores=properatti_df["price"].value_counts(dropna=False)
contar_valores

In [ ]:
#Tipos de monedas que componen el campo currency#

In [ ]:
df_currency_type = properatti_df.groupby("currency")

In [ ]:
df_currency_type_values = list(df_currency_type.groups.keys())
df_currency_type_values

In [ ]:
#Cantidad de entries con cada campo#

precio_por_moneda=properatti_df.groupby("currency")["price"].count()
precio_por_moneda

In [ ]:
mascara_uyu=precios["currency"]=="UYU"
fila_uy=precios[mascara_uyu]
fila_uy

In [ ]:
mascara_pen=precios["currency"]=="PEN"
fila_pen=precios[mascara_pen]
fila_pen

In [ ]:
valores=properatti_df[mascara_uyu]
pd.set_option("display.max_colwidth",1000)
print(valores["description"])
print(valores["title"])

In [ ]:
valor_pesos=precios[mascara_uyu]
valor_pesos

In [ ]:
#Filtramos los datos de descripcion por moneda#

import re

In [ ]:
data_filtrada = properatti_df[properatti_df['price'].isnull()]
regex_pattern_2 = re.compile(r'\b(?:usd|us\$|u\$s|\$|ars)\b', re.IGNORECASE)
filtered_df_2 = data_filtrada[
    data_filtrada["description"].str.contains(regex_pattern_2) |
    data_filtrada["title"].str.contains(regex_pattern_2)
]

print(filtered_df_2.shape)

hay 3695 lineas filtradas que corresponden a valores de precio nulos y que se pueden rellenar
#

In [ ]:
# Define a function to extract substrings around the regex pattern
def extract_substring(text, regex_pattern):
    match = re.search(regex_pattern, text)
    if match:
        start_index = max(0, match.start() - 5)
        end_index = match.end() + 10
        return text[start_index:end_index]
    else:
        return ""



# Use .loc to set values in the DataFrame
print(filtered_df_2.shape)
filtered_df_2.loc[:, 'description_short'] = filtered_df_2['description'].apply(lambda x: extract_substring(x, regex_pattern_2))
filtered_df_2.loc[:, 'title_short'] = filtered_df_2['title'].apply(lambda x: extract_substring(x, regex_pattern_2))
print(filtered_df_2.shape)
print(filtered_df_2["description"])
print(*filtered_df_2["title_short"])



In [ ]:
filtered_df_2.dtypes

In [ ]:
#Luego de filtrados los datos por moneda, los filtramos por moneda de vuelta, por . y por cantidad de 5 digitos o màs (para descartar que se pusieran datos que correspondan a expensas u otras cosas )

pattern_regex="U\$S\s|usd|us\$*"
dolar_regex=re.compile(pattern_regex)
decimal_regex="\."
compile_decimal=re.compile(decimal_regex)

sin_dolar = filtered_df_2.description_short.apply(lambda x: x if x is np.NaN else dolar_regex.sub("", x))
sin_dolar.head(10)
sin_punto=sin_dolar.apply(lambda x: x if x is np.NaN else compile_decimal.sub("", x))
sin_punto.head(10)

pattern_numero=r'(\d{5,})'
regex_numero=re.compile(pattern_numero)
cinco_digitos=sin_punto.str.extract(pattern_numero).astype(float)
print(cinco_digitos)
cinco_digitos.isnull().count()


In [ ]:
#En base a los anteriores filtrados y a descartar la cantidad de entries que no cumplen con el campo quedan 580 entries en descripcion que permiten llenar el campo precio#

In [ ]:
nulos_descripcion=cinco_digitos.dropna()

In [ ]:
print(nulos_descripcion.count())


In [ ]:
precios["price_clean"]=precios["price"].copy()

In [ ]:
precios["price_clean"].head(19)

In [ ]:
print(nulos_descripcion[0])

In [ ]:
precios['Clean_precios'] = precios["price_clean"].fillna(nulos_descripcion.iloc[:,0])

In [ ]:
precios["Clean_precios"]

In [ ]:
nulos_descripcion.iloc[:,0]

In [ ]:
pattern_regex="U\$S\s|usd|us\$*"
dolar_regex=re.compile(pattern_regex)
decimal_regex="\."
compile_decimal=re.compile(decimal_regex)

sin_dolar = filtered_df_2.title_short.apply(lambda x: x if x is np.NaN else dolar_regex.sub("", x))
sin_dolar.head(10)
sin_punto=sin_dolar.apply(lambda x: x if x is np.NaN else compile_decimal.sub("", x))
sin_punto.head(10)

pattern_numero=r'(\d{5,})'
regex_numero=re.compile(pattern_numero)
cinco_digitos_ti=sin_punto.str.extract(pattern_numero).astype(float)
print(cinco_digitos_ti)
cinco_digitos_ti.isnull().count()

In [ ]:
#Se hizo el mismo protocolo para el titulo (el ti de los comandos hace referencia a eso) se encontraron 2412 campos que permitirian llenar#

In [ ]:
nulos_descripcion_ti=cinco_digitos_ti.dropna()
nulos_descripcion_ti.count()

In [ ]:
precios['Clean_precio_titulo'] = precios["Clean_precios"].fillna(nulos_descripcion_ti.iloc[:,0])

In [ ]:
precios["Clean_precio_titulo"].isnull().sum()

In [ ]:
#mismo protocolo para el caso de la URL. se encontraron 0 campos que permite llenar#


sin_punto=sin_dolar.apply(lambda x: x if x is np.NaN else compile_decimal.sub("", x))
sin_punto.head(10)

pattern_numero=r'(\d{5,})'
regex_numero=re.compile(pattern_numero)
cinco_digitos_url=sin_punto.str.extract(pattern_numero).astype(float)
print(cinco_digitos_url)
cinco_digitos_url.isnull().count()
nulos_descripcion_url=cinco_digitos_url.dropna()
nulos_descripcion_url.count()

Una vez hecho eso, pensaba en funcion de los campos
price, price_aprox_local_currency, price_aprox_USD y currency estimar  un promedio de la cotizacion del dolar
para, de esa forma, poder normalizar todas las cotizaciones a USD

para hacer lo anterior hago una mascara con currency ars

In [ ]:
mascara_ars=precios["currency"]=="ARS"
entries_en_ARS=precios[mascara_ars]
entries_en_ARS.head(10)

In [ ]:
precios["conversion_en_USD"]=entries_en_ARS["price_aprox_local_currency"]/entries_en_ARS["price_aprox_usd"]

In [ ]:
precios["conversion_en_USD"].head(10)

In [ ]:
descarto_nulos_conversion_USD=precios["conversion_en_USD"].dropna()

In [ ]:
descarto_nulos_conversion_USD.head(10)

In [ ]:
descarto_nulos_conversion_USD.mean()

In [ ]:
#De acuerdo con este promedio 1 dolar corresponde a 17,6445 pesos Se podrìa comprobar lo mismo con las entradas correspondientes al campo currency USD

In [ ]:
mascara_usd=precios["currency"]=="USD"
entries_en_usd=precios[mascara_usd]
entries_en_usd.head(10)



In [ ]:
precios["conversion_en_USD_currency_USD"]=entries_en_usd["price_aprox_local_currency"]/entries_en_usd["price_aprox_usd"]

In [ ]:
descarto_nulos_conversion_USD_currency_USD=precios["conversion_en_USD_currency_USD"].dropna()

In [ ]:
descarto_nulos_conversion_USD_currency_USD.head(10)

In [ ]:
descarto_nulos_conversion_USD_currency_USD.mean()

In [ ]:
#De acuerdo con este promedio 1 dolar corresponde a 17,6445 pesos

#Valor con las entradas correspondientes al campo currency USD
#Se pueden normalizar los entries por ese valor de pesos

In [ ]:
#conversion de los campos del dataframe PRecios que tiene como moneda ARS a USD
#a la vez se corrigen los valores de ARS#

In [ ]:
precios.loc[mascara_ars,"Clean_precio_titulo"]/=(descarto_nulos_conversion_USD.mean())

In [ ]:
precios[mascara_ars]

In [ ]:
precios["currency_new"]=precios["currency"].replace("ARS","USD")

In [ ]:
precios["currency_new"]

In [ ]:
precios = precios.rename(columns={'Clean_precio_titulo': 'precios_nuevos'})

In [ ]:
precios.dtypes

Cambio todos los valores del campo currency_new a USD#

In [ ]:
fill="USD"

In [ ]:
precios['currency_new'] = precios.apply(lambda row: fill if pd.isna(row['currency_new']) and pd.notna(row['precios_nuevos']) else row['currency_new'], axis=1)

Por ùltimo tomo en cuenta los valores de los campos correspondientes a las sntradas de currency en UYU y en PEN
y las normalizo a USD

In [ ]:
precios[mascara_uyu]

In [ ]:
fila_uy["price_aprox_local_currency"]/fila_uy["price_aprox_usd"]
masc_uyu=properatti_df["currency"]=="UYU"
properatti_df[masc_uyu]

In [ ]:
fila_uy["price_aprox_local_currency"]/fila_uy["price"]
fila_uy["price"]/fila_uy["price_aprox_usd"]

In [ ]:
new_price_UYUtoUSD=fila_uy["price_aprox_usd"]
new_price_UYUtoUSD

In [ ]:
precios["precios_nuevos"][107391]=new_price_UYUtoUSD

In [ ]:
precios["precios_nuevos"][107391]

In [ ]:
fila_pen

In [ ]:
fila_pen["price"]/fila_pen["price_aprox_usd"]

In [ ]:
fila_pen["price_aprox_local_currency"]/fila_pen["price_aprox_usd"]

In [ ]:
new_price_PENtoUSD=fila_pen["price_aprox_usd"]

In [ ]:
precios.loc[mascara_pen,"precios_nuevos"]=new_price_PENtoUSD

In [ ]:
precios['currency_new'].replace("PEN","USD")
precios["currency_new"].replace("UYU","USD")
precios["currency_new"]

Luego de seleccionadas las columnas correpsondeintes a price aprox en USD para los casos de pesos UYU y de PEN se
unificaron los currency en la columna currency para establecer su normalizacion a USD

In [ ]:
precios.dtypes

In [ ]:
precios=precios.drop(labels=["price_clean", "Clean_precios", "conversion_en_USD", "conversion_en_USD_currency_USD"], axis=1)

In [ ]:
precios

In [ ]:
print("nulos en la columna precios nueva :", precios['precios_nuevos'].isnull().sum())
print("nulos en la columna precios inicial:", precios['price'].isnull().sum())
properatti_df['precios_nuevos'] = precios['precios_nuevos']

filtered_df_sum = properatti_df[(properatti_df['precios_nuevos'].isna())].shape[0]
print(filtered_df_sum)

In [ ]:
properatti_df["precios_nuevos"].shape[0]

In [ ]:
no_nulos_precios_usd=properatti_df["price_aprox_usd"].notnull().sum()
no_nulos_precios_usd

In [ ]:
no_nulos_precios_nuevos=properatti_df["precios_nuevos"].notnull().sum()
no_nulos_precios_nuevos

In [ ]:
data_filtrada2 = properatti_df[properatti_df['precios_nuevos'].notnull()]

In [ ]:
print(data_filtrada2["price_aprox_usd"].isnull().sum())
print(data_filtrada2["surface_total_clean"].isnull().sum())

In [ ]:
#Filtro los datos por precio para corroborar que este bien hecho
data_filtrada3=properatti_df[properatti_df["price"].notnull()]
print(data_filtrada3["price_aprox_usd"].isnull().sum())
print(data_filtrada3["price_aprox_local_currency"].isnull().sum())
print(data_filtrada3["price_per_m2"].isnull().sum())
print(data_filtrada3["price_usd_per_m2"].isnull().sum())
print(data_filtrada3["surface_total_clean"].isnull().sum())

In [ ]:
#Lleno los datos faltantes correspondientes a los precios en usd y en local currency
properatti_df['precios_aprox_usd_clean'] = properatti_df["price_aprox_usd"].fillna(properatti_df.loc[:,"precios_nuevos"])

properatti_df['precios_aprox_local_currency_clean'] = properatti_df["price_aprox_usd"].fillna(properatti_df.loc[:,"precios_nuevos"]*(17.6445))

In [ ]:
#corroboro el correcto llenado de la nueva columna precios_aprox_usd_clean
data_filtrada4=properatti_df[properatti_df["precios_nuevos"].notnull()]
print(data_filtrada4["precios_aprox_usd_clean"].isnull().sum())
data_filtrada4['precios_aprox_local_currency_clean'].isnull().sum()

In [ ]:
# filtro por la columna superficie limpiada
data_filtrada5=properatti_df[properatti_df["surface_total_clean"].notnull()]
print(data_filtrada5["precios_nuevos"].isnull().sum())
print(data_filtrada5["price_per_m2"].isnull().sum())
print(data_filtrada5["price_usd_per_m2"].isnull().sum())
print(data_filtrada5["price_usd_per_m2"].notnull().sum())

In [ ]:
#filtro este dataframe datafiltrada5 por price_aprox_usd-notnull

In [ ]:
data_filtrada_6=data_filtrada5[data_filtrada5["price_per_m2"].notnull()]
print(data_filtrada_6.shape[0])
print(data_filtrada_6["precios_nuevos"].isnull().sum())
data_filtrada_6=data_filtrada5[data_filtrada5["price_usd_per_m2"].notnull()]
print(data_filtrada_6.shape[0])
print(data_filtrada_6["precios_nuevos"].isnull().sum())

#no hay campos NaN en precios nuevos que puedan ser llenados con datos correspondientes a las otras dos columnas

In [ ]:
"No puedo llenar el dataframe con datos que esten en la columna surface_total_clean y que esten en price_per_m2 o price_usd_per_m2"

In [ ]:
# tomo la data filtrada 2 (la hecha a partir de precios nuevos y filtro por price_usd_per_m2. Se busca rellenar la columna surface_total_clean)

filtro_precio_por_m2=data_filtrada2[data_filtrada2["price_usd_per_m2"].notnull()]
filtro_precio_por_m2["surface_total_clean"].isnull().sum()

# No puedo llenar ningùn campo correspondiente a surface_total_clean dado que en ka data filtrada, no presenta valores nulos

# Floor (Valentino)

In [ ]:
df_floor=properatti_df.floor
#creamos variable de la columna floor

In [ ]:
df_floor.size
#tamanio de la columna

In [ ]:
df_floor.isnull().sum()
#cantidad de nulos

In [ ]:
df_floor.value_counts(bins=[1,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80])
#cantidad por rango

In [ ]:
df_floor.loc[df_floor>=75].value_counts()

In [ ]:
df_floor_clean=df_floor.apply(lambda x: np.nan if x>=54 or x==np.nan else x)
#eliminamos los valores mayores a 54

In [ ]:
#ponemos valor 100 para cuando no tiene valor la casilla
#esto para cuando se trate de una casa que no tiene piso o por si tenia un valor irreal
df_floor_clean=df_floor_clean.fillna(100)

In [ ]:
# A continuacion vemos que valores de propiedad se toman cuando no es nulo el valor floor
properatti_df.loc[properatti_df.floor>=0]['property_type'].value_counts()
# Concluimos en que los valores de apartment estan bien ya que excluimos aquellos mayores a 54
# Pero no seria logico que el valor de una casa un PH o una store sea mayor a 1 entoces esos valores hay que pasarlos a 100 (nuestro sinonimo de nulo)

In [ ]:
indice=properatti_df.loc[properatti_df.floor>1][properatti_df.property_type!='apartment']['floor'].index
indice

In [ ]:
df_floor_clean.iloc[indice]=100
df_floor_clean.value_counts()

In [ ]:
null_count = df_floor_clean.isnull().sum()
print(null_count)

# Limpieza de la columna "expenses"

In [ ]:
#creo el dataframe expensas para analizar
expensas=properatti_df.loc[:,["expenses", "description", "title", "properati_url"]]
expensas.head(4)

In [ ]:
#se calcula la cantidad de nulos en la columna expensas
cant_nulos_expensas=expensas["expenses"].isnull().sum()

In [ ]:
cant_nulos_expensas

In [ ]:
cantidad_entries=expensas.shape[0]

In [ ]:
cantidad_entries

In [ ]:
#se calcula el porcentaje de nulos
porcentaje_nulos_expensas=cant_nulos_expensas/cantidad_entries*100

In [ ]:
porcentaje_nulos_expensas.round(2)

In [ ]:
expensas.dtypes

In [ ]:
#busco los campos expensas que se encuentren en las columnas description, title y url
data_filtrada = expensas[expensas["expenses"].isnull()]
regex_pattern_exp = re.compile(r'\b(exp*)\b', re.IGNORECASE)
filtered_df_2 = data_filtrada[
    data_filtrada["description"].str.contains(regex_pattern_exp) |
    data_filtrada["title"].str.contains(regex_pattern_exp)
]

print(filtered_df_2.shape)

In [ ]:
def extract_substring(text, regex_pattern):
    match = re.search(regex_pattern, text)
    if match:
        start_index = max(0, match.start() - 5)
        end_index = match.end() + 10

        return text[start_index:end_index]
    else:
        return ""



# Use .loc to set values in the DataFrame
print(filtered_df_2.shape)
filtered_df_2.loc[:, 'description_short'] = filtered_df_2['description'].apply(lambda x: extract_substring(x, regex_pattern_exp))
filtered_df_2.loc[:, 'title_short'] = filtered_df_2['title'].apply(lambda x: extract_substring(x, regex_pattern_exp))
print(filtered_df_2.shape)
print(filtered_df_2["description"])
print(*filtered_df_2["title_short"])


In [ ]:
#calculo el porcentaje del campo expensas que quedaría sin llenar
cantidad_maxima_a_agregar=(filtered_df_2.shape[0])/cantidad_entries*100
campos_sin_llenar_en_porcentaje=porcentaje_nulos_expensas-cantidad_maxima_a_agregar
campos_sin_llenar_en_porcentaje.round(0)
calculate_correlation(properatti_df[properatti_df['property_type']=='apartment'],'rooms','surface_covered_in_m2')


Los campos que contienen exp (expenses) que se extraen de la descripcion, de title y de url son, en total, a lo sumo 689.
Representa menos del 0,6% de campos totales. Por lo tanto, quedan alrededor de 88 % de campos sin llenar.

---


Se descarta la columna





# Amenities

In [ ]:
#Definir dictionario de amenities
sum_sinonimos={
    'sum': 'sum',
    's.u.m': 'sum',
    'salón de fiestas':'sum',
    'salón de usos múltiples':'sum'}
estacionamiento_sinonimos={
    'estacionamiento':'estacionamiento',
    'cochera':'estacionamiento',
    'garage':'estacionamiento',
    'cocheras':'estacionamiento'}
pileta_sinonimos= {
    'pileta': 'pileta',
    'piscina':'pileta'}
parrilla_sinonimos= {
    'parrilla':'parrilla',
    'parrillas':'parrilla',
    'asador':'parrilla'}
gimnasio_sinonimos= {
    'gym':'gimnasio',
    'gimnasio':'gimnasio'}
lavadero_sinonimos= {
    'laundry':'lavadero',
    'lavadero':'lavadero'}
solarium_sinonimos= {
    'solárium':'solarium',
    'solarium':'solarium'}
quincho_sinonimos= {
    'quincho':'quincho'}
baulera_sinonimos= {
    'baulera':'baulera'}
terraza_sinonimos= {
    'terraza':'terraza'}

amenities=['sum','estacionamiento','pileta','parrilla','gimnasio','lavadero','solarium','quincho','baulera','terraza']

In [ ]:
#Normalizando la columna description
properatti_df['description']=properatti_df['description'].str.lower()

In [ ]:
#Funcion para reemplazar los sinonimos de los amenities
def amenitie_standard (df,synoms):
    df.replace(synoms,regex=True,inplace=True)
    return df

In [ ]:
#Crear una columna que defina si tiene alguno de los amenities previamente definidos
properatti_df['amenities']=properatti_df['description'].str.contains('|'.join(amenities))

In [ ]:
properatti_df["estacionamiento"]=(amenitie_standard (properatti_df["description"],estacionamiento_sinonimos)).str.contains('estacionamiento')
properatti_df["sum"]=(amenitie_standard (properatti_df["description"],sum_sinonimos)).str.contains('sum')
properatti_df["pileta"]=(amenitie_standard (properatti_df["description"],pileta_sinonimos)).str.contains('pileta')
properatti_df["parrilla"]=(amenitie_standard (properatti_df["description"],parrilla_sinonimos)).str.contains('parrilla')
properatti_df["gimnasio"]=(amenitie_standard (properatti_df["description"],gimnasio_sinonimos)).str.contains('gimnasio')
properatti_df["lavadero"]=(amenitie_standard (properatti_df["description"],lavadero_sinonimos)).str.contains('lavadero')
properatti_df["solarium"]=(amenitie_standard (properatti_df["description"],solarium_sinonimos)).str.contains('solarium')
properatti_df["quincho"]=(amenitie_standard (properatti_df["description"],quincho_sinonimos)).str.contains("quincho")
properatti_df["baulera"]=(amenitie_standard (properatti_df["description"],baulera_sinonimos)).str.contains("baulera")
properatti_df["terraza"]=(amenitie_standard (properatti_df["description"],terraza_sinonimos)).str.contains("terraza")

In [ ]:
#Validar la creacion de las nuevas columnas
properatti_df.columns

In [ ]:
#Pocentaje de propiedades con amenities
amenities= properatti_df[properatti_df['amenities']==True]
amenities['amenities'].count()/properatti_df['amenities'].count()*100

In [ ]:
#Porcentaje de tipos de propiedades con amenities
amenities['property_type'].value_counts()/properatti_df['property_type'].value_counts()*100

# Correlaciones

In [ ]:
#filtro el dataframe por provincia
grouped_df = properatti_df.groupby('provincia').agg({'precios_nuevos': 'mean', 'surface_total_clean': 'mean'}).reset_index()

#creo el gráfico de dispersión utilizando los datos agrupados
plt.scatter(grouped_df['surface_total_clean'], grouped_df['precios_nuevos'])
plt.title('Diagrama de Dispersión Promedio entre Precio y Superficie Total por Ubicación')
plt.xlabel('Superficie Total Promedio por Ubicación')
plt.ylabel('Precio Promedio por Ubicación')
plt.xlim(0, 700)
plt.ylim(0, 600000)
plt.show()

In [ ]:
#correlaciones de precio con las variables superficie, rooms y precio por m2


In [ ]:
#Creo una mascara para cada una de las provincias
masks = [
    properatti_df["provincia"]=='Buenos Aires',
    properatti_df["provincia"]=='Cordoba',
    properatti_df["provincia"]=='Santa Fe',
    properatti_df["provincia"]=='Rio Negro',
    properatti_df["provincia"]=='Neuquen',
    properatti_df["provincia"]=='Tucuman',
    properatti_df["provincia"]=='Mendoza',
    properatti_df["provincia"]=='Corrientes',
    properatti_df["provincia"]=='Misiones',
    properatti_df["provincia"]=='Entre Rios',
    properatti_df["provincia"]=='Salta',
    properatti_df["provincia"]=='Chubut',
    properatti_df["provincia"]=='San Luis',
    properatti_df["provincia"]=='La Pampa',
    properatti_df["provincia"]=='Formosa',
    properatti_df["provincia"]=='Chaco',
    properatti_df["provincia"]=='San Juan',
    properatti_df["provincia"]=='Tierra Del Fuego',
    properatti_df["provincia"]=='Catamarca',
    properatti_df["provincia"]=='Jujuy',
    properatti_df["provincia"]=='Santa Cruz',
    properatti_df["provincia"]=='Santiago Del Estero',
    properatti_df["provincia"]=='La Rioja']
# Variables para calcular la correlación
variable1 = 'precios_nuevos'
variable2 = 'surface_total_clean'
variable3 = "price_usd_per_m2"
variable4 = "rooms_clean"

# Se Crean las listas para guardar los resultados y un bucle for para aplicar las máscaras y calcular la correlación
lista_correlacion1=[]
lista_correlacion2=[]
lista_correlacion3=[]
for i, mask in enumerate(masks):
    # Se Aplica la máscara al DataFrame
    filtered_df = properatti_df[mask]

    # Se calcula la correlación entre dos variables especificadas
    correlation1 = calculate_correlation (filtered_df, 'precios_nuevos', 'surface_total_clean')
    correlation2 = calculate_correlation (filtered_df, 'precios_nuevos', 'price_usd_per_m2')
    correlation3 = calculate_correlation (filtered_df, 'precios_nuevos', 'rooms_clean')

    lista_correlacion1.append(correlation1)
    lista_correlacion2.append(correlation2)
    lista_correlacion3.append(correlation3)


# Se imprime la correlación para cada máscara


print(lista_correlacion1)
print(lista_correlacion2)
print(lista_correlacion3)

In [ ]:
#se crea una lista con los nombres de las provincias
lista_provincias= [
    'Buenos Aires',
    'Cordoba',
    'Santa Fe',
    'Rio Negro',
    'Neuquen',
    'Tucuman',
    'Mendoza',
    'Corrientes',
    'Misiones',
    'Entre Rios',
    'Salta',
    'Chubut',
    'San Luis',
    'La Pampa',
    'Formosa',
    'Chaco',
    'San Juan',
    'Tierra Del Fuego',
    'Catamarca',
    'Jujuy',
    'Santa Cruz',
    'Santiago Del Estero',
    'La Rioja']

In [ ]:
#se crean tres diccionarios, se los transforma en 3 dataframes. Luego se copian las columnas de correlacion al primer dataframe
diccionario_correlacion_precio_superficie_en_m2=dict(zip(lista_provincias,lista_correlacion1))
diccionario_correlacion_precio_precio_usd_por_m2=dict(zip(lista_provincias,lista_correlacion2))
diccionario_correlacion_precio_rooms=dict(zip(lista_provincias,lista_correlacion3))

correlacion1_df=pd.DataFrame(list(diccionario_correlacion_precio_superficie_en_m2.items()), columns=["Provincia", "Correlacion_superficie"])


correlacion2_df=pd.DataFrame(list(diccionario_correlacion_precio_precio_usd_por_m2.items()), columns=["Provincia", "Correlacion_precio_usd_por_m2"])


correlacion3_df=pd.DataFrame(list(diccionario_correlacion_precio_rooms.items()), columns=["Provincia", "Correlacion_rooms"])



correlacion1_df["Correlacion_precio_usd_por_m2"]=correlacion2_df["Correlacion_precio_usd_por_m2"]
correlacion1_df["Correlacion_rooms"]= correlacion3_df["Correlacion_rooms"]
correlacion1_df

In [ ]:
#grafico para comparar los nulos antes y después de la limpieza

lista4 = ['precios_nuevos', 'price', 'surface_total_clean', 'surface_total_in_m2', 'rooms_clean', 'rooms', 'precios_aprox_usd_clean', 'price_aprox_usd']

valores4 = [properatti_df['precios_nuevos'].isnull().sum()/(properatti_df.shape[0])*100,
            properatti_df['price'].isnull().sum()/(properatti_df.shape[0])*100,
            properatti_df['surface_total_clean'].isnull().sum()/(properatti_df.shape[0])*100,
            properatti_df['surface_total_in_m2'].isnull().sum()/(properatti_df.shape[0])*100,
            properatti_df['rooms_clean'].isnull().sum()/(properatti_df.shape[0])*100,
            properatti_df['rooms'].isnull().sum()/(properatti_df.shape[0])*100,
            properatti_df['precios_aprox_usd_clean'].isnull().sum()/(properatti_df.shape[0])*100,
            properatti_df['price_aprox_usd'].isnull().sum()/(properatti_df.shape[0])*100, ]

colores = ['blue', 'green', 'blue', 'green', 'blue', 'green', 'blue', 'green']

fig, ax = plt.subplots()

for i in range(len(lista4)):
    ax.bar(i, valores4[i], color=colores[i], label=lista4[i])

ax.set(xlabel="Columna", ylabel="Porcentaje de nulos", title="Compararión de porcentaje de nulos")
plt.xticks(np.arange(len(lista4)), lista4, rotation=45, ha="right")
plt.show()